# Deep learning with Differential Privacy - Main script. Author: Reinier Vos (4663160 TUD) Maciej Jankowski (4651294 TUD)

## Imports

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass
  
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Normalization

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from tqdm import tqdm # gives progress bar when loading
#import tensorflow_datasets as tfds
import matplotlib.ticker as mticker
import scipy.stats as sc
import time
from sklearn.preprocessing import StandardScaler
import random
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
# verbosity 
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.autograph.set_verbosity(0)

# Script Main Parameters

In [ ]:
# main
seed = 2
std_pca = 7 #16 #4 # std for pca
std_sgd = 4 # std for dp_sgd
batch_size = 600 
lr_sgd = 0.05 # [0.01,0.07] stable, best at 0.05
C = 4 # gradient clipping bound
gs = batch_size

# moment accountant specific
parameters_ma = {"maxOrder":32,
                 "sigma": std_sgd,
                 "q": batch_size/60000,
                 "T":400}
debug = True
#'''
deltaFixed = False
epsFixed= True
epsilon = 2
th_delta = 100#10**-5 # epsilon fixed
'''
deltaFixed = True 
epsFixed= False
delta = 10e-5
th_epsilon = 2 # delta fixed
'''
allParameters = {**parameters_ma,
                'seed' : seed,
                'std_pca' :std_pca, 
                'std_sgd' : std_sgd,
                'batch_size' : batch_size, 
                'lr_sgd' : lr_sgd, 
                'C' : C, 
                'deltaFixed' : deltaFixed, 
                'epsFixed': epsFixed, 
                'epsilon' : epsilon, 
                'th_delta' : th_delta, 
}
'''
allParameters = {**parameters_ma,
                'seed' = seed,
                'std_pca' =std_pca, 
                'std_sgd' = std_sgd,
                'batch_size' = batch_size, 
                'lr_sgd' = lr_sgd, 
                'C' = C, 
                'deltaFixed' = deltaFixed, 
                'epsFixed'= epsFixed, 
                'delta' = delta, 'th_epsilon' = th_epsilon, 
}
'''
batches = 100
np.random.seed(seed)  

## Load and Preprocess Data
We frist load the MNIST dataset using Tensorflow Datasets. This dataset has 28 x 28 grayscale images of digits belonging to 10 classes.


In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
# reshape
x_train = x_train.reshape(x_train.shape[0],784)
x_test = x_test.reshape(x_test.shape[0],784)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

covar = tf.cast(tf.einsum('ji,jk->ik', x_train, x_train), 'float32')
shape = tf.shape(covar)
noise = tf.random.normal(shape, mean=0, stddev=16, dtype=tf.dtypes.float32)
noise = (noise + noise.T)/2
covar += noise # covariance matrix used for pca

In [ ]:
# get batches
indices = np.arange(0,len(x_train))
np.random.shuffle(indices)

x_train_batches = np.array_split(x_train[indices,:], batches)
y_train_batches = np.array_split(y_train[indices], batches)
train = list(zip(x_train_batches, y_train_batches))
x_test_batches = np.array_split(x_test, batches)
y_test_batches = np.array_split(y_test, batches)
test = list(zip(x_test_batches, y_test_batches))

In [ ]:
class_names = ["0","1","2","3","4","5","6","7","8","9","10"]

Next, you normalize the images by dividing them by 255.0 so as to make the pixels fall in the range (0, 1). You also reshape the data so as to flatten the 28 x 28 pixel array into a flattened 784 pixel array.

Now you shuffle and batch your training and test datasets before feeding them to the model.

## Define the Model - DP for MNIST as in paper


### DP PCA layer

In [ ]:
class DP_PCA(tf.keras.layers.Layer):
    def __init__(self, pca_components, seed, std, covar):
        super(DP_PCA, self).__init__()
        self.pca_components = pca_components
        self.seed = seed
        self.std = std
        self.covar = covar

    def build(self, input_shape):
        a = 1 # placeholder

    def call(self, inputs):
        #inputs = tf.linalg.normalize(inputs, ord=2, axis=-1)
        #inputs = tf.math.l2_normalize(inputs, axis=-1)
        #covar = tf.einsum('ji,jk->ik', inputs, inputs)
        #print(tf.shape(covar))
        # add noise
        covar = self.covar # copy
        #shape = tf.shape(covar)
        #noise = tf.random.normal(shape, mean = 0, stddev = self.std, dtype=tf.dtypes.float32, seed=self.seed)
        #noise = np.tril(noise) + np.tril(noise, -1).T
        #noise = noise + noise.T - tf.linalg.diag_part(noise)
        #noise = (noise + noise.T)/2
        #print(noise == noise.T)
        #covar += noise
        # add vectors
        e_values, e_vectors = tf.linalg.eigh(covar)
        return tf.einsum('ij,jk->ik', inputs, e_vectors[:,-self.pca_components:])
        #return tf.einsum('ji,ik->jk', inputs, e_vectors[:,-self.pca_components:])

### Base model

In [ ]:
def base_model():
    inputs = tf.keras.Input(shape=(784,), name='digits')
    x = DP_PCA(60, seed, std_pca, covar)(inputs)
    x = tf.keras.layers.Dense(1000, activation='relu', name='dense_1')(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax', name='predictions')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.summary()
    return model

## Define Optimizer and Loss Function


In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate = lr_sgd) # [0.01,0.07] stable, best at 0.05
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.compat.v1.losses.Reduction.NONE)


## Define Metrics & Loss functions

In [ ]:
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

## Moment accountant class

In [ ]:
class moment_accountant():
    def __init__(self,seed, params: dict,deltaFixed = False, epsFixed= False, debug = False, **kwargs):
        
        # moment accountant hyperparameters
        self.mixN = 1000 # samples for mixture of moment arrays
        self.debug = debug
        self.seed = seed
        
        # constants 
        self.maxOrder = params["maxOrder"] # maximum moment order 
        self.lambd = np.arange(0,self.maxOrder+1)
        #self.lambd = np.array([0])
        self.lambdaN = len(self.lambd) # number of lambdas to check
        print(f"Maxorder = {self.maxOrder}, with order array:")
        print(self.lambd)
        self.sigma = params["sigma"]
        self.q = params["q"]
        self.T = params["T"]
        
        # booleans 
        self.deltaFixed = deltaFixed
        self.epsFixed = epsFixed
        
        if self.deltaFixed:
            if self.epsFixed:
                raise Exception("Choose ONLY epsilon or delta as fixed")
            # in case delta is held fixed
            print("keeping delta fixed")
            self.delta = kwargs["delta"]
            self.th_epsilon = kwargs["th_epsilon"]
        elif self.epsFixed:
            # in case epsilon is held fixed
            print("keeping epsilon fixed")
            self.epsilon = kwargs["epsilon"]
            self.th_delta = kwargs["th_delta"]
        else:
            raise Exception("Choose EITHER epsilon or delta as fixed")
        
        #self.e1_mu0, self.e1_mu, self.e2_mu0, self.e2_mu = self._setup_mixNormNP() # obtain random sample arrays NUMPY
        self.e1_mu0, self.e1_mu, self.e2_mu0, self.e2_mu = self._setup_mixNormTF() # obtain random sample arrays TENSORFLOW
        self.alpha = self._compute_moment(self.lambd)
        
        # initializations 
        self.alphaSum = 0 # moment
        self.lambdArgmin = []
        self.iterations = 0
        self.deltaList = []
        self.epsList = []
        # =================================
        print("moment accountant setup complete")
        
    def _setup_mixNormTF(self):
        '''
        Mixture of gaussians by tensorflow, so no assumptions used
        https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/Mixture
        '''
        
        '''
        CHECKED - WORKING CORRECTLY
        '''
        # setup normal dis & mixture of normals arrays
        np.random.seed(self.seed) # set seed
        mu_0 = np.random.normal(0, self.sigma, (self.mixN))
        # analytical mean & var for mix
        
        # setup gaussian mixture
        dismix_mu = tfp.distributions.Mixture(cat=tfp.distributions.Categorical(probs=[1.-self.q, self.q]),
                                    components=[tfp.distributions.Normal(loc=0., scale=self.sigma),
                                                tfp.distributions.Normal(loc=1., scale=self.sigma)])
        mu = dismix_mu.sample(sample_shape=(self.mixN), seed=self.seed).numpy()

        # find pdf values for z's 
        e1_mu0 = sc.norm.pdf(mu_0,loc = 0, scale = self.sigma)
        e1_mu = dismix_mu.prob(mu_0).numpy()
        
        e2_mu0 = sc.norm.pdf(mu,loc = 0, scale = self.sigma)
        e2_mu = dismix_mu.prob(mu).numpy()

        return e1_mu0, e1_mu, e2_mu0, e2_mu
                            
    def _compute_moment(self, lambd: np.array):
        '''
        CHECKED - WORKING CORRECTLY
        '''
        # computes unbiased expectation for E1 & E2, then the moment alpha 
        lambd = np.broadcast_to(np.expand_dims(lambd, -1), (self.lambdaN, self.mixN)) # broadcast
        #E1 = 1/self.mixN*np.sum(np.transpose(np.power(np.transpose(self.mu_0/self.mu),lambd)), axis = 0)
        #E2 = 1/self.mixN*np.sum(np.transpose(np.power(np.transpose(self.mu/self.mu_0),lambd)), axis = 0)
        E1 = np.nanmean(np.transpose(np.power(np.transpose(self.e1_mu0/self.e1_mu),lambd)), axis = 0)
        '''
        note that due to setup E1 will always be < 1 since denom > num always
        '''
        E2 = np.nanmean(np.transpose(np.power(np.transpose(self.e2_mu/self.e2_mu0),lambd)), axis = 0)
        '''
        note that due to setup E2 will always be > 1 since denom < num always
        '''
        #alpha = np.log(np.maximum(E1,E2))
        alpha = np.log(np.maximum(E1,E2))
        return alpha
    
    def compute_deltaEps(self):
        # tail bound
        #alpha = self._compute_moment(self.lambd) + self.alpha # note that this is the log moment!
        alpha = self.alphaSum + self.alpha # note that this is the log moment!
        self.alphaSum = alpha # update moment
        if self.epsFixed:
            # epsilon is kept fixed, compute delta
            epsilon = self.epsilon
            delta = np.min(np.exp(alpha-self.lambd*epsilon))
            # TODO remove inf or nan <- does not seem necessary
            if self.debug:
                ind = np.argmin(np.exp(alpha-self.lambd*epsilon))
                self.lambdArgmin.append(self.lambd[ind])
        if self.deltaFixed:
            # delta is kept fixed, compute epsilon
            delta = self.delta
            epsilon = (alpha-np.log(delta))/self.lambd
            if self.debug:
                ind = np.argmin(epsilon)
                self.lambdArgmin.append(self.lambd[ind])
            epsilon = np.min(epsilon)
            # TODO remove inf or nan <- does not seem necessary
        self.epsList.append(epsilon)
        self.deltaList.append(delta)
        return delta, epsilon
    
    def check_thresholds(self, delta: float, epsilon: float):
        go = True
        if self.epsFixed:
            if self.th_delta < delta: 
                # delta threshold exceeded
                go = False
        if self.deltaFixed:
            if self.th_epsilon < epsilon:
                # epsilon threshold exceeded
                go = False
        return go
    
    def plot_traces(self):
        if len(self.epsList) == 0:
            raise Exception("Apply iterations on accountant instance before calling this function")
        elif not self.debug:
            raise Exception("Debug was set to false, thus not all relevant data was collected")
        else:
            # gather data
            epsilon = np.array(self.epsList)
            delta = np.array(self.deltaList)
            lambdas = np.array(self.lambdArgmin)
            print(f"Delta fixed = {self.deltaFixed}| Last delta = {delta[-1]}")
            print(f"Epsilon fixed = {self.epsFixed}| Last epsilon = {epsilon[-1]}")
            print("Fixed parameter will not be plotted \n NOTE: Iteration arrays are returned")
            # plotting
            iterations = np.arange(0,len(epsilon))
            plt.figure()
            if self.deltaFixed:
                plt.plot(iterations,epsilon, label = 'epsilon')
            else:
                plt.plot(iterations,delta, label = 'delta')
            #plt.legend(loc='upper left')
            plt.title("epsilon or delta over iterations")
            
            plt.figure()
            plt.plot(iterations,lambdas)
            plt.title("Lambda chosen over iterations")
            
        return delta, epsilon, lambdas


## Differential privary variant of SGD


In [ ]:
class DP_SGD:
    def __init__(self, lr = 0.01, sigma = 0.2, gs = 10, C = 1, seed = 2): # TODO; check defaults
        self.lr = lr # learning rate
        self.sigma = sigma # sigma, noise scale
        self.gs = gs # group size
        self.C = C # gradient norm bound
        self.seed = seed
        self.std = sigma*C # for noise addition
        #self.loss_object = ? # define this here
        
    def apply_gradients(self, optimizer, model, loss_object, x, y):

        #n = tf.shape(x).numpy()
        ## run loop
        with tf.GradientTape(persistent = True) as tape:
            y_pred = model(x)
            loss = loss_object(y, y_pred)
            loss_red = tf.reduce_sum(loss, axis = 0)
            
            
        #########
        # MAIN PROBLEM: I only get a single set of gradients even if i have a loss function which outputs loss for every sample
        # do you want me to parallelize this myself
        ########
        ## obtain gradients    
        #grad = tape.batch_jacobian(loss, model.trainable_weights)
        grad = tape.jacobian(loss, model.trainable_weights, parallel_iterations = None, experimental_use_pfor= False)
        
        ## clip gradients per layer
        for l in range(len(grad)):
            #clipper = tf.norm(grad[l], ord = 2, axis = 0)
            dims = len(tf.shape(grad[l]))
            clipper = tf.math.square(grad[l])
            
            if dims > 2:
                # kernel layers
                clipper = tf.reduce_sum(clipper, axis = [1,2])
            else:
                # bias layer
                clipper = tf.reduce_sum(clipper, axis = -1)
                
            clipper = tf.math.sqrt(clipper)
            clipper = tf.math.maximum(tf.constant([1], dtype = tf.dtypes.float32),clipper/self.C)
            if dims > 2:
                # kernel layers
                clipper = tf.broadcast_to(tf.expand_dims(tf.expand_dims(clipper, -1), -1),tf.shape(grad[l]))
            else:
                # bias layer
                clipper = tf.broadcast_to(tf.expand_dims(clipper, -1),tf.shape(grad[l]))
            grad[l] = tf.math.divide(grad[l],clipper) # override
                
        ## add noise
        for l in range(len(grad)): # loop over layers
            grad_red = tf.math.reduce_sum(grad[l], axis = 0)
            shape = tf.shape(grad_red)
            noise = tf.random.normal(shape, mean = 0, stddev = self.std, dtype=tf.dtypes.float32, seed=self.seed)
            grad[l] = tf.add(grad_red, noise)/self.gs
            


        ## descent
        #print(grad)
        '''
        print(f'HELOO={len(grad)}')
        print(f'HELOO={len(model.trainable_weights)}')
        ga = []
        ma = []
        print(type(model.trainable_weights))
        for l in range(len(model.trainable_weights)):
            ga.append(tf.shape(grad[l]).numpy())
            ma.append(tf.shape(model.trainable_weights[l]).numpy())
        zipp = list(zip(ga,ma))
        print(zipp)
        
        #print(model.trainable_weights)
        '''
        
        optimizer.apply_gradients(zip(grad,model.trainable_weights))
            
        ## collect
    
        return y_pred,loss_red 
        
        

## Building Training Loop

In [ ]:
def apply_gradient(optimizer, model, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss_value = loss_object(y_true=y, y_pred=logits)

    gradients = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights)) # ZIP ENSURES THAT GRADIENTS IS APPLIED TO EVERY LAYER CORRECTLY (SINCE EVERY LAYER HAS W & b != not 1 variable)

    return logits, loss_value

In [ ]:
def train_data_for_one_epoch(dp_sgd, optimizer, model, loss_object, moment_accountant):
    losses = []
    pbar = tqdm(total=len(list(enumerate(train))), position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} ') # progress bar
    go = True
    step = 0 
    for x_batch_train, y_batch_train in train:
        step += 1
        logits, loss_value = dp_sgd.apply_gradients(optimizer, model, loss_object, x_batch_train, y_batch_train)
        #logits, loss_value = apply_gradient(optimizer, model, x_batch_train, y_batch_train)
        delta, eps = moment_accountant.compute_deltaEps()
        
        losses.append(loss_value)

        train_acc_metric(y_batch_train, logits)
        pbar.set_description("Training loss for step %s: %.4f" % (int(step), float(loss_value)))
        pbar.update()
        
        if not moment_accountant.check_thresholds(delta, epsilon):
            go = False
            break
    print(f"Delta = {delta} | Epsilon = {epsilon}")
    return losses, go 

At the end of each epoch you have to validate the model on the test dataset. The following function calculates the loss on test dataset and updates the states of the validation metrics.

In [ ]:
def perform_validation():
    losses = []
    for x_val, y_val in test:
        val_logits = model(x_val)
        val_loss = loss_object(y_val, val_logits)
        #val_loss = tf.reduce_sum(val_loss, axis = 0).numpy()
        losses.append(val_loss)
        val_acc_metric(y_val, val_logits)
    return losses

In [ ]:
## INITIALIZE 
model = base_model()
model.layers[2].trainable = False # pca layer
DPSGD = DP_SGD(lr_sgd,std_sgd, gs, C, seed)
if epsFixed: 
    print("\n Epsilon kept fixed \n")
    accountant = moment_accountant(seed, parameters_ma, deltaFixed = deltaFixed, epsFixed= epsFixed, debug = debug, epsilon = epsilon, th_delta = th_delta)
else:
    print("\n Delta kept fixed \n")
    #delta fixed
    accountant = moment_accountant(seed, parameters_ma, deltaFixed = deltaFixed, epsFixed= epsFixed, debug = debug, delta = delta, th_epsilon = th_epsilon)

# Iterate over epochs.
epochs = 125 #18
epochs_val_losses, epochs_train_losses = [], []
val_acc_list, train_acc_list = [], []
for epoch in range(epochs):
    print('Start of epoch %d' % (epoch,))

    losses_train, go = train_data_for_one_epoch(DPSGD, optimizer, model, loss_object, accountant)
    train_acc = train_acc_metric.result()

    losses_val = perform_validation()
    val_acc = val_acc_metric.result()

    train_acc_list.append(train_acc)
    val_acc_list.append(val_acc)

    losses_train_mean = np.mean(losses_train)
    losses_val_mean = np.mean(losses_val)
    epochs_val_losses.append(losses_val_mean)
    epochs_train_losses.append(losses_train_mean)

    print('\n Epoch %s: Train loss: %.4f  Validation Loss: %.4f, Train Accuracy: %.4f, Validation Accuracy %.4f' % (epoch, float(losses_train_mean), float(losses_val_mean), float(train_acc), float(val_acc)))

    train_acc_metric.reset_states()
    val_acc_metric.reset_states()
    if not go:
        print(f"\n Stopping due to privacy loss at epoch {epoch}/{epochs} \n")
        break


## Evaluate Model

In [ ]:
with open(f'accuracies', 'wb') as f:
    np.save(f, np.array(val_acc_list))
    np.save(f, np.array(train_acc_list))
#np.load(accuracies)
#val_acc_list, train_acc_list

### Plots for Evaluation

In [ ]:
def plot_graph(train_acc,val_acc,deltas):

  fig, ax1 = plt.subplots()
  x = np.arange(1,len(train_acc)+1)
  ax1.plot(x,train_acc,'bs',label='training accuracy')
  ax1.plot(x,val_acc,'r',label='validation accuracy')
  ax1.set_ylim([0.7,1])
  ax1.set_xticks(np.arange(0, max(x)+1, 10.0))
  ax1.set_xlabel('Epochs' )
  ax1.set_ylabel('accuracy',color='r')
  ax1.legend()
  ax2 = ax1.twinx()
  ax2.yaxis.tick_right()
  ax2.yaxis.set_label_position('right')
  ax2.xaxis.tick_top()
  ax2.xaxis.set_label_position('top')

  ax2.plot(x,deltas, marker = 'x', color = 'black')
  #ax2.set_ylim([0,1])
  ax2.set_xlabel('epsilon')
  ax2.set_ylabel('deltas')

  plt.show()




plot_graph(train_acc_list,val_acc_list,delta)

In [ ]:
def plot_metrics(train_metric, val_metric, metric_name, title, ylim=5):
    plt.title(title)
    #plt.ylim(0,ylim)
    plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
    x = np.arange(1,len(train_metric)+1)
    plt.plot(x,train_metric,color='blue',label=metric_name)
    plt.plot(x,val_metric,color='green',label='val_' + metric_name)
    plt.legend()

plot_metrics(epochs_train_losses, epochs_val_losses, "Loss", "Loss", ylim=10.0)